Скрипт выполняет ручную дедупликацию названий районов на основе предопределенной карты DUPLICATE_MAP, которая сопоставляет неправильные названия с их каноническими версиями. Для каждой пары он сначала обновляет внешний ключ (big_folk_district_id) в таблице buildings, перенаправляя ссылки с дубликата на канонический ID, а затем удаляет устаревшую (дублирующую) запись из таблицы big_folk_districts, обеспечивая единообразие данных.

In [1]:
import sqlite3
import pandas as pd


DB_NAME = '../databases/dataset.db'
# Таблица, которую мы чистим (например, big_folk_districts или folk_districts)
DISTRICTS_TABLE = 'big_folk_districts' 
# Столбец в таблице 'buildings', который ссылается на DISTRICTS_TABLE
FOREIGN_KEY_COLUMN = 'big_folk_district_id' 

# Список дубликатов и канонических названий:
# { 'Неправильное_Имя': 'Каноническое_Имя', ... }
DUPLICATE_MAP = {
    'о. Русский': 'О. Русский',
    'о. Попова': 'О. Попова',
    'Вторая речка': 'Вторая Речка',
    '64 71 микрорайон': '64, 71 микрорайоны', # Выбираем одно из возможных написаний
    '3 рабочая': 'Третья рабочая' # Исправление опечатки
    # Если есть еще дубликаты 'Весенняя', добавьте их здесь, если ID разные
}

# 2. Установление соединения и определение функций
try:
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    print(f"Успешное подключение к базе данных: {DB_NAME}")

    def get_id_by_name(name, table):
        """Возвращает ID по названию района."""
        query = f"SELECT id FROM {table} WHERE name = ?"
        cursor.execute(query, (name,))
        result = cursor.fetchone()
        return result[0] if result else None

    def consolidate_duplicates(duplicate_map, districts_table, fk_column):
        """Основная функция для объединения дубликатов."""
        print(f"\n--- Начинается консолидация в таблице {districts_table} ---")
        
        for wrong_name, correct_name in duplicate_map.items():
            
            # Получаем ID канонического и дублирующего района
            correct_id = get_id_by_name(correct_name, districts_table)
            wrong_id = get_id_by_name(wrong_name, districts_table)
            
            if not correct_id:
                print(f"❌ Ошибка: Не найден канонический ID для '{correct_name}'. Пропуск.")
                continue
            
            if not wrong_id:
                print(f"✅ Предупреждение: Не найден дубликат '{wrong_name}'. Возможно, уже исправлено. Пропуск.")
                continue

            print(f"\nОбработка: '{wrong_name}' (ID: {wrong_id}) -> '{correct_name}' (ID: {correct_id})")
            
            # 1. Обновление ссылок в таблице buildings
            update_query = f"""
            UPDATE buildings
            SET {fk_column} = ? 
            WHERE {fk_column} = ?
            """
            cursor.execute(update_query, (correct_id, wrong_id))
            print(f"   -> Обновлено ссылок в таблице buildings: {conn.total_changes}")

            # 2. Удаление дублирующей записи из таблицы районов
            delete_query = f"""
            DELETE FROM {districts_table} WHERE id = ?
            """
            cursor.execute(delete_query, (wrong_id,))
            print(f"   -> Удален дубликат из таблицы {districts_table}.")

        print("\n--- Консолидация завершена. ---")


    # 3. Выполнение очистки
    
    # 🚨 ВНИМАНИЕ: Запустите эту функцию, чтобы применить изменения.
    consolidate_duplicates(DUPLICATE_MAP, DISTRICTS_TABLE, FOREIGN_KEY_COLUMN)

    # 4. Сохранение изменений
    conn.commit()
    print("\nИзменения сохранены в базе данных.")

except sqlite3.Error as e:
    print(f"Произошла ошибка SQLite: {e}")
finally:
    if conn:
        conn.close()
        print("Соединение с БД закрыто.")

Успешное подключение к базе данных: ../databases/dataset.db

--- Начинается консолидация в таблице big_folk_districts ---

Обработка: 'о. Русский' (ID: 009af94d0a44e256f651e1f85d1b94e3) -> 'О. Русский' (ID: bbc3aa6c1924ba7aa571cb4d588355d3)
   -> Обновлено ссылок в таблице buildings: 839
   -> Удален дубликат из таблицы big_folk_districts.

Обработка: 'о. Попова' (ID: 16aa0efc7fbf0eacc272f84b771a8327) -> 'О. Попова' (ID: 807df8befe5a46cb79c5398a0760afc4)
   -> Обновлено ссылок в таблице buildings: 1084
   -> Удален дубликат из таблицы big_folk_districts.

Обработка: 'Вторая речка' (ID: 28f0c59576ad51c52bf864a76670abc2) -> 'Вторая Речка' (ID: 3caec8129b7983068459b162184fabe8)
   -> Обновлено ссылок в таблице buildings: 2634
   -> Удален дубликат из таблицы big_folk_districts.

Обработка: '64 71 микрорайон' (ID: b7cb6104dd1d8aa6c8752b879da04eee) -> '64, 71 микрорайоны' (ID: d3f85944a9834555a85268dd33c7a448)
   -> Обновлено ссылок в таблице buildings: 2651
   -> Удален дубликат из таблицы

Скрипт выполняет базовую очистку таблицы связей blackouts_buildings путём удаления всех записей, где внешний ключ blackout_id имеет значение NULL. Это обеспечивает целостность связи "отключение-здание", устраняя некорректные записи, которые не связаны ни с одним фактическим отключением.

In [7]:
import sqlite3

# 1. Параметры подключения к базе данных
DB_NAME = '../databases/dataset.db'
TABLE_TO_CLEAN = 'blackouts_buildings'
COLUMN_TO_CHECK = 'blackout_id'

conn = None # Инициализируем соединение
try:
    # 2. Установление соединения
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    print(f"Успешное подключение к базе данных: {DB_NAME}\n")
    
    # Сначала считаем, сколько записей будет удалено
    count_query = f"""
    SELECT COUNT(*) FROM {TABLE_TO_CLEAN} WHERE {COLUMN_TO_CHECK} IS NULL;
    """
    cursor.execute(count_query)
    records_to_delete = cursor.fetchone()[0]
    
    if records_to_delete == 0:
        print(f"✅ В таблице {TABLE_TO_CLEAN} не найдено записей с {COLUMN_TO_CHECK} = NULL. Очистка не требуется.")
    else:
        print(f"Обнаружено записей для удаления: {records_to_delete}")
        
        # 3. Выполнение запроса на удаление
        delete_query = f"""
        DELETE FROM {TABLE_TO_CLEAN} WHERE {COLUMN_TO_CHECK} IS NULL;
        """
        cursor.execute(delete_query)
        
        # 4. Сохранение изменений
        conn.commit()
        
        print(f"🗑️ Успешно удалено {conn.total_changes} записей из таблицы {TABLE_TO_CLEAN}.")
        print("\nИзменения сохранены в базе данных.")

except sqlite3.Error as e:
    print(f"❌ Произошла ошибка SQLite: {e}")
finally:
    # 5. Закрытие соединения
    if conn:
        conn.close()
        print("Соединение с БД закрыто.")

Успешное подключение к базе данных: ../databases/dataset.db

Обнаружено записей для удаления: 1243966
🗑️ Успешно удалено 1243966 записей из таблицы blackouts_buildings.

Изменения сохранены в базе данных.
Соединение с БД закрыто.


Скрипт проводит комплексную диагностику чистоты данных в таблице streets, проверяя на NULL-значения и настоящие дубликаты (одинаковое название улицы в пределах одного города после нормализации). Дополнительно он проверяет ссылочную целостность, выявляя здания в таблице buildings, которые ссылаются на несуществующие street_id, предоставляя рекомендации для устранения всех найденных проблем.

In [3]:
import sqlite3
import pandas as pd

DB_NAME = '../databases/dataset.db'
STREETS_TABLE = 'streets'
BUILDINGS_TABLE = 'buildings'
CITIES_TABLE = 'cities'

conn = None
try:
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # --- Подготовка: Получение справочника городов для удобного вывода ---
    df_cities = pd.read_sql_query(f"SELECT id, name FROM {CITIES_TABLE}", conn)
    city_map = df_cities.set_index('id')['name'].to_dict()
    print(f"Успешное подключение к базе данных: {DB_NAME}")
    print(f"Обнаруженные города: {list(city_map.values())}")
    
    # Загрузка таблицы streets в DataFrame
    df_streets = pd.read_sql_query(f"SELECT id, name, city_id FROM {STREETS_TABLE}", conn)
    
    
    # ----------------------------------------------------------------------
    ## 1. Проверка на NULL в ключевых полях
    # ----------------------------------------------------------------------
    print("\n" + "="*50)
    print("## 1. Проверка на NULL в streets")
    print("="*50)
    
    null_counts = df_streets.isnull().sum()
    
    if null_counts.sum() == 0:
        print("✅ NULL не обнаружено ни в одном столбце.")
    else:
        print("❌ Обнаружены NULL:")
        print(null_counts[null_counts > 0])
        print("Рекомендация: записи с пустым 'id' или 'name' следует удалить.")

    
    # ----------------------------------------------------------------------
    ## 2. Проверка НАСТОЯЩИХ дубликатов (Одинаковое название И Город)
    # ----------------------------------------------------------------------
    print("\n" + "="*50)
    print("## 2. Проверка НАСТОЯЩИХ дубликатов")
    print("   (Одинаковое название в ОДНОМ городе)")
    print("="*50)
    
    # Нормализуем название улицы для поиска дубликатов без учета регистра/пробелов
    df_streets['normalized_name'] = df_streets['name'].str.lower().str.strip()
    
    # Ищем дубликаты, которые имеют одинаковое нормализованное название И одинаковый city_id
    duplicate_streets = df_streets.duplicated(subset=['normalized_name', 'city_id'], keep=False)
    duplicate_names = df_streets[duplicate_streets]
    
    if duplicate_names.empty:
        print("✅ Потенциальные дубликаты (одинаковое название в ОДНОМ городе) не обнаружены. Таблица чиста!")
    else:
        # Группируем, чтобы увидеть дублирующиеся пары
        duplicates_grouped = duplicate_names.groupby(['normalized_name', 'city_id'])[['id', 'name']].agg(list)
        duplicates_grouped['ID_Count'] = duplicates_grouped['id'].apply(len)
        
        print(f"❌ Обнаружено {len(duplicates_grouped)} пар, которые являются НАСТОЯЩИМИ дубликатами.")
        
        # Выводим результат с названием города
        output_df = duplicates_grouped.reset_index().drop(columns=['normalized_name'])
        output_df['City_Name'] = output_df['city_id'].map(city_map)
        
        print(output_df[['City_Name', 'name', 'ID_Count']])
        
        print("\nРекомендация: Выберите один канонический ID для каждого дубликата и обновите ссылки в 'buildings', затем удалите дубликаты из 'streets'.")
        
    
    # ----------------------------------------------------------------------
    ## 3. Проверка распределения по городам (city_id)
    # ----------------------------------------------------------------------
    print("\n" + "="*50)
    print("## 3. Проверка распределения по городам")
    print("="*50)

    # Группируем улицы по city_id
    city_counts = df_streets['city_id'].map(city_map).value_counts(dropna=False)
    
    print("Распределение улиц по городам:")
    print(city_counts)
    
    # Находим ID Владивостока
    vladivostok_id = df_cities[df_cities['name'].str.contains('Владивосток', case=False)]['id'].iloc[0] if not df_cities[df_cities['name'].str.contains('Владивосток', case=False)].empty else None
    


    # ----------------------------------------------------------------------
    ## 4. Проверка ссылочной целостности: Buildings -> Streets
    # ----------------------------------------------------------------------
    print("\n" + "="*50)
    print("## 4. Проверка ссылочной целостности: Buildings -> Streets")
    print("="*50)
    
    # Находим buildings, которые ссылаются на несуществующий street_id
    orphan_query = f"""
    SELECT 
        COUNT(T1.id) 
    FROM {BUILDINGS_TABLE} AS T1
    LEFT JOIN {STREETS_TABLE} AS T2 ON T1.street_id = T2.id
    WHERE T2.id IS NULL AND T1.street_id IS NOT NULL;
    """
    
    cursor.execute(orphan_query)
    orphan_count = cursor.fetchone()[0]
    
    if orphan_count == 0:
        print("✅ Нет зданий, ссылающихся на несуществующие street_id.")
    else:
        print(f"❌ Обнаружено {orphan_count} зданий с недействительным street_id.")
        print("Рекомендация: Эти 'street_id' в таблице buildings следует либо обнулить, либо найти/создать соответствующие записи в таблице streets.")


except sqlite3.Error as e:
    print(f"❌ Произошла ошибка SQLite: {e}")
finally:
    if conn:
        conn.close()
        print("\nСоединение с БД закрыто.")

Успешное подключение к базе данных: ../databases/dataset.db
Обнаруженные города: ['Артем', 'Владивосток']

## 1. Проверка на NULL в streets
✅ NULL не обнаружено ни в одном столбце.

## 2. Проверка НАСТОЯЩИХ дубликатов
   (Одинаковое название в ОДНОМ городе)
✅ Потенциальные дубликаты (одинаковое название в ОДНОМ городе) не обнаружены. Таблица чиста!

## 3. Проверка распределения по городам
Распределение улиц по городам:
city_id
Владивосток    1201
Артем           822
Name: count, dtype: int64

## 4. Проверка ссылочной целостности: Buildings -> Streets
✅ Нет зданий, ссылающихся на несуществующие street_id.

Соединение с БД закрыто.


Скрипт выполняет дедупликацию названий районов в таблице folk_districts путём сопоставления дубликатов (например, разных написаний одного и того же места) с каноническими именами с использованием регулярных выражений и нормализации. После идентификации дубликатов, он обновляет соответствующие внешние ключи в таблице buildings и удаляет лишние записи из folk_districts, обеспечивая целостность и чистоту данных.

In [2]:
import sqlite3
import pandas as pd
import re

# 1. Параметры подключения
DB_NAME = '../databases/dataset.db'
FD_TABLE = 'folk_districts'
FOREIGN_KEY_COLUMN = 'folk_district_id' 

# 🚨 Канонические названия, которые ДОЛЖНЫ остаться
# Каноническое имя для SPECIAL_GROUPS (П-ов. Песчаный) должно быть здесь, 
# чтобы его ID был гарантированно найден.
CANONICAL_NAMES = [
    '64, 71 микрорайон',
    'О. Попова',
    'о. Рейнеке',
    'О. Русский',
    'П-ов. Песчаный',  # ← Каноническое имя для всех вариантов Песчаного
    'Район Хребта Богатая Грива',
    'Тавайза',
    'Угловое'
]

# 🔥 СПЕЦИАЛЬНЫЕ ГРУППЫ ДЛЯ СЛОЖНЫХ СЛУЧАЕВ
SPECIAL_GROUPS = {
    'Песчаный': {
        'canonical': 'П-ов. Песчаный', # Указывает на имя из CANONICAL_NAMES
        'patterns': [
            # Нормализация для REGEX: все в нижнем регистре, без лишних символов
            r'п[\s\-]*ов\.?\s*песчан',     # п-ов. Песчаный, пов. Песчаный
            r'полуостров\s*песчан',       # полуостров Песчаный
            r'по[\s\-]*ов\.?\s*песчан',     # по-ов. Песчаный
            r'песчан+ый',                 # Песчаный, Песчанный
        ]
    }
}

def normalize_name(name):
    """
    Универсальная функция нормализации: 
    - Нижний регистр
    - Удаление точек, запятых, дефисов, пробелов
    """
    if pd.isna(name):
        return None
    
    return (
        str(name).lower()
        .replace('.', '').replace(',', '').replace(' ', '')
        .replace('-', '')
    )

# --- Основная логика ---
conn = None
try:
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    print(f"✅ Успешное подключение к базе данных: {DB_NAME}")
    
    # 1. Загрузка и подготовка данных
    df_fd = pd.read_sql_query(f"SELECT id, name FROM {FD_TABLE}", conn)
    print(f"📊 Загружено записей из {FD_TABLE}: {len(df_fd)}")

    # Создаем нормализованную колонку для универсальной проверки
    df_fd['normalized_name'] = df_fd['name'].apply(normalize_name)
    
    # 2. Формирование карты замены ID
    ID_REPLACEMENT_MAP = {}
    total_duplicates_found = 0
    canonical_ids = {} # Карта для хранения найденных канонических ID

    # --- Находим все канонические ID в первую очередь ---
    for c_name in CANONICAL_NAMES:
        row = df_fd[df_fd['name'] == c_name]
        if row.empty:
            print(f"❌ Ошибка: Не найден канонический ID для '{c_name}'. Проверьте список CANONICAL_NAMES.")
            continue
        canonical_ids[c_name] = row['id'].iloc[0]

    if not canonical_ids:
        raise Exception("Не найдено ни одного канонического ID. Проверьте имена в базе.")

    # ----------------------------------------------------------------------
    # 2.1. Обработка специальных групп (Regex)
    # ----------------------------------------------------------------------
    processed_ids = set() # Чтобы не обрабатывать один ID дважды
    for group_name, group_config in SPECIAL_GROUPS.items():
        canonical_name = group_config['canonical']
        patterns = group_config['patterns']
        
        if canonical_name not in canonical_ids:
            continue # Ошибка уже была выведена выше

        correct_id = canonical_ids[canonical_name]
        print(f"\n🔍 Специальная группа '{group_name}' (ID: {correct_id})")
        
        matching_ids = []
        for pattern in patterns:
            # Используем .str.contains() с скомпилированным паттерном для поиска в name
            compiled_pattern = re.compile(pattern, re.IGNORECASE)
            # Фильтруем по паттерну, исключая канонический ID и уже обработанные
            matches = df_fd[
                (df_fd['name'].str.contains(compiled_pattern, na=False)) & 
                (df_fd['id'] != correct_id)
            ]
            
            for wrong_id in matches['id'].tolist():
                if wrong_id not in ID_REPLACEMENT_MAP and wrong_id not in processed_ids:
                    ID_REPLACEMENT_MAP[wrong_id] = correct_id
                    processed_ids.add(wrong_id)
                    total_duplicates_found += 1
                    # Находим имя по ID для лога
                    dup_name = df_fd[df_fd['id'] == wrong_id]['name'].iloc[0]
                    print(f"   🎯 Найден дубликат по REGEX '{pattern}': '{dup_name}' (ID: {wrong_id})")

        print(f"   ✅ Найдено дубликатов в группе: {len([k for k, v in ID_REPLACEMENT_MAP.items() if v == correct_id])}")


    # ----------------------------------------------------------------------
    # 2.2. Обработка остальных канонических имён (Нормализация)
    # ----------------------------------------------------------------------
    print("\n🔍 --- Обработка остальных канонических имен (Нормализация) ---")
    
    for canonical_name in CANONICAL_NAMES:
        correct_id = canonical_ids.get(canonical_name)
        if not correct_id:
            continue # Ошибка уже была выведена

        # Пропускаем те, которые уже обработаны (или будут обработаны) в спецгруппах
        if any(canonical_name == group_config['canonical'] for group_config in SPECIAL_GROUPS.values()):
            continue
            
        normalized_key = normalize_name(canonical_name)
        
        # Ищем все записи, нормализованное имя которых совпадает с нормализованным каноническим
        group_rows = df_fd[df_fd['normalized_name'] == normalized_key]
        
        # Берем ID, которые не являются каноническим и еще не были обработаны в спецгруппах
        ids_to_delete = group_rows[
            (group_rows['id'] != correct_id) &
            (~group_rows['id'].isin(processed_ids))
        ]['id'].tolist()
        
        if not ids_to_delete:
            print(f"   ✅ '{canonical_name}': Дубликатов не найдено.")
            continue
            
        # Заполняем карту замены
        for wrong_id in ids_to_delete:
            ID_REPLACEMENT_MAP[wrong_id] = correct_id
        
        duplicates_count = len(ids_to_delete)
        total_duplicates_found += duplicates_count
        print(f"   ✅ '{canonical_name}' (ID: {correct_id}): Найдено {duplicates_count} дубликатов → {ids_to_delete}")
    
    
    print(f"\n📈 ИТОГО: Найдено {total_duplicates_found} дубликатов для замены!")
    print(f"📋 Карта замены (Wrong ID → Correct ID): {ID_REPLACEMENT_MAP}")
    
    if not ID_REPLACEMENT_MAP:
        print("\n🎉 Дубликатов не найдено! База уже чистая.")
    else:
        # ----------------------------------------------------------------------
        # 3. Выполнение замены и удаления (массовые запросы)
        # ----------------------------------------------------------------------
        print("\n🔄 --- Выполнение замены и удаления в БД (Массово) ---")
        
        # Группируем ID для замены по каноническому ID
        updates_to_run = {}
        for wrong_id, correct_id in ID_REPLACEMENT_MAP.items():
            if correct_id not in updates_to_run:
                updates_to_run[correct_id] = []
            updates_to_run[correct_id].append(wrong_id)

        total_buildings_updated = 0
        ids_to_delete_list = list(ID_REPLACEMENT_MAP.keys())
        
        # A. Обновление ссылок в таблице buildings
        for correct_id, wrong_ids in updates_to_run.items():
            wrong_ids_str = ','.join(map(str, wrong_ids))
            
            update_query = f"""
            UPDATE buildings
            SET {FOREIGN_KEY_COLUMN} = ? 
            WHERE {FOREIGN_KEY_COLUMN} IN ({wrong_ids_str})
            """
            cursor.execute(update_query, (correct_id,))
            updated_count = cursor.rowcount
            total_buildings_updated += updated_count
            print(f"   📍 Обновлено зданий для ID {correct_id}: {updated_count}")
        
        # B. Удаление дублирующих записей
        ids_to_delete_str = ','.join(map(str, ids_to_delete_list))
        delete_query = f"""
        DELETE FROM {FD_TABLE} WHERE id IN ({ids_to_delete_str})
        """
        cursor.execute(delete_query)
        deleted_count = cursor.rowcount
        print(f"   🗑️  Удалено дубликатов из {FD_TABLE}: {deleted_count}")
        
        # 4. Сохранение изменений
        conn.commit()
        print(f"\n🎉 ✅ ИТОГО:")
        print(f"   📍 Обновлено ссылок в buildings: {total_buildings_updated}")
        print(f"   🗑️  Удалено дубликатов из {FD_TABLE}: {deleted_count} (ожидалось {len(ID_REPLACEMENT_MAP)})")
        print("   💾 Изменения СОХРАНЕНЫ в базе данных!")

except sqlite3.Error as e:
    print(f"❌ Произошла ошибка SQLite: {e}")
    if conn:
        conn.rollback()
        print("   🔄 Изменения отменены (rollback).")
except Exception as e:
    print(f"❌ Неожиданная ошибка: {e}")
    if conn:
        conn.rollback()
        print("   🔄 Изменения отменены (rollback).")
finally:
    if conn:
        conn.close()
        print("🔌 Соединение с БД закрыто.")

✅ Успешное подключение к базе данных: ../databases/dataset.db
📊 Загружено записей из folk_districts: 3594

🔍 Специальная группа 'Песчаный' (ID: 2b045aed62a3ca490fde36fdad2b78f2)
   🎯 Найден дубликат по REGEX 'п[\s\-]*ов\.?\s*песчан': 'п-ов Песчаный' (ID: 496a98094feab6c1daf1c6803dcfde7e)
   🎯 Найден дубликат по REGEX 'по[\s\-]*ов\.?\s*песчан': 'По-ов. Песчанный' (ID: 11e0f52adfe4a253cf3cc1d48dd12802)
   🎯 Найден дубликат по REGEX 'по[\s\-]*ов\.?\s*песчан': 'По-ов. Песчанный' (ID: 39a7baf6077f13db7888cf7a82abeff2)
   🎯 Найден дубликат по REGEX 'по[\s\-]*ов\.?\s*песчан': 'По-ов. Песчанный' (ID: 2c4e3fe794239840f5d2897e0fe8a500)
   🎯 Найден дубликат по REGEX 'по[\s\-]*ов\.?\s*песчан': 'По-ов. Песчанный' (ID: f4edc4a3567f88dd657841c099200682)
   🎯 Найден дубликат по REGEX 'по[\s\-]*ов\.?\s*песчан': 'По-ов. Песчанный' (ID: fa547163c8738a3e4dd3400e15b4e5fb)
   ✅ Найдено дубликатов в группе: 6

🔍 --- Обработка остальных канонических имен (Нормализация) ---
   ✅ '64, 71 микрорайон' (ID: a5af42